# Assignment 5

**Assignment 5: Neuroevolution: Neural Architecture**

**Goal**: Get familiar with how neuroevolution is used for neural network architecture optimization by optimizing a neural network using neuroevolution.

## 1 Understanding the problem



*Problem description goes here*

In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt